In [ ]:
!mkdir analysis

mkdir: cannot create directory ‘analysis’: File exists


In [ ]:
!pip install geopandas

In [ ]:
# import required libraries
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import statistics

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# uncomment to unzip London bourough shape files
#!unzip 'gdrive/My Drive/data/statistical-gis-boundaries-london.zip'
# loading shape file with geodata for London boroughs
fp = '/content/statistical-gis-boundaries-london/ESRI/London_Borough_Excluding_MHW.shp'
gdf = gpd.read_file(fp)

Archive:  gdrive/My Drive/data/statistical-gis-boundaries-london.zip
replace __MACOSX/._statistical-gis-boundaries-london? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
gdf.head()

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
0,Kingston upon Thames,E09000021,3726.117,0.000,F,None,None,"POLYGON ((516401.600 160201.800, 516407.300 16..."
1,Croydon,E09000008,8649.441,0.000,F,None,None,"POLYGON ((535009.200 159504.700, 535005.500 15..."
2,Bromley,E09000006,15013.487,0.000,F,None,None,"POLYGON ((540373.600 157530.400, 540361.200 15..."
3,Hounslow,E09000018,5658.541,60.755,F,None,None,"POLYGON ((521975.800 178100.000, 521967.700 17..."
4,Ealing,E09000009,5554.428,0.000,F,None,None,"POLYGON ((510253.500 182881.600, 510249.900 18..."


In [ ]:
# discarding unnecesary data
gdf = gdf[['NAME', 'geometry','HECTARES']]
gdf = gdf.rename(columns={'NAME':'borough_name'})
gdf.head()

,borough_name,geometry,HECTARES
0,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117
1,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441
2,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487
3,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541
4,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428


In [ ]:
# load infrastructure data
venues = pd.read_csv('gdrive/My Drive/data/all_venues.csv')
venues.head()

,Unnamed: 0,name,borough_name,longitude,latitude,borough_code,website,address1,address2,address3,os_addressbase_uprn,ward_2018_name,ward_2018_code,easting,northing,runtime
0,0,Abacus Arts,Southwark,-0.095863,51.489864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Academy Performing Arts,Greenwich,0.068762,51.494097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Alford House,Lambeth,-0.114177,51.487992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Alleyn's School,Southwark,-0.081993,51.454781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Anatomie Studio,Southwark,-0.070739,51.482080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# discard unnecesary columns and replace ensure consistent naming with the geodata
venues = venues[['name','borough_name','longitude','latitude']]
venues['borough_name'] = venues['borough_name'].str.replace('City of Westminster', 'Westminster')
venues['borough_name'] = venues['borough_name'].str.replace('City and County of the City of London', 'City of London')

In [ ]:
# merge datasets
merged = pd.merge(venues, gdf,
                  on='borough_name',
                  how='left')

In [ ]:
# add column for number of venues in each borough
gdf['venue_count'] = np.nan

# determine number of venues in each borough and populate the venue_count column accordingly
for index, row in gdf.iterrows():
  borough = row['borough_name']
  count = (merged[merged.borough_name == borough].shape[0])
  gdf.loc[index, 'venue_count'] = count

In [ ]:
# populate venue_density column (no. of venues divided by area)
gdf['venue_density'] = gdf['venue_count']/gdf['HECTARES']
gdf.head()

,borough_name,geometry,HECTARES,venue_count,venue_density
0,Kingston upon Thames,"POLYGON ((516401.600 160201.800, 516407.300 16...",3726.117,15.0,0.004026
1,Croydon,"POLYGON ((535009.200 159504.700, 535005.500 15...",8649.441,35.0,0.004047
2,Bromley,"POLYGON ((540373.600 157530.400, 540361.200 15...",15013.487,43.0,0.002864
3,Hounslow,"POLYGON ((521975.800 178100.000, 521967.700 17...",5658.541,39.0,0.006892
4,Ealing,"POLYGON ((510253.500 182881.600, 510249.900 18...",5554.428,31.0,0.005581


In [ ]:
!pip install -q geopy

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
# method matches a named location (string) with coordinates for that location ([latitude, longitude] pair)
def query_locations(query):
  geolocator = Nominatim(user_agent="my_request")
  location = geolocator.geocode(query)
  return [location.latitude, location.longitude]

In [ ]:
# creating density map
# source: https://towardsdatascience.com/visualizing-bike-mobility-in-london-using-interactive-maps-for-absolute-beginners-3b9f55ccb59

# create the basemap
London = [51.506949, -0.122876]
map = folium.Map(location = London, 
                zoom_start = 12, 
                tiles = "CartoDB dark_matter")

# plot circle for each borough with radius size corresponding to venue_density measure
for index, row in gdf.iterrows():
  location = query_locations("london borough of " + row['borough_name'])
  venue_density = row['venue_density']
  radius = (venue_density)*2000
  if np.isnan(radius):
    radius = 0,
  popup = str(row['borough_name'])+': ' + str(int(row['venue_count'])) + ' venues \n area: ' + str(int(row['HECTARES'])) + ' hectares'

  folium.CircleMarker(location = location, 
                    radius = radius, 
                    popup = popup, 
                    color = '#E80018', 
                    fill_opacity = 0.5).add_to(map)

In [ ]:
# export map as html file
map.save('map_density_done.html')